In [1]:
from selenium import webdriver
from shutil import which
from selenium.webdriver.chrome.options import Options
import re
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from tqdm import tqdm

class EMSI_skills:
    def open_driver(self,path, headless=False, incognito=True):
        
        #Open chrome driver
        chrome_options = Options()
        
        #Headless mode
        if headless==True:
            chrome_options.add_argument("--headless")
            
        #incognito mode
        if incognito == True:
            chrome_options.add_argument("--incognito")
        
        #open the driver and go to linkedin login page
        driver = webdriver.Chrome(executable_path=path, options=chrome_options)
        driver.set_page_load_timeout(30)
        return driver
    
    #Not used
    def login(self, driver, login="", password=""):
        login = "rvaladares@neobrain.io"
        password = "dsnblisboa2020"
        #Login on LinkedIn
        driver.get("https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fsearch%2Fresults%2Fcompanies%2F%3Fkeywords%3DApple%3DGLOBAL_SEARCH_HEADER&amp;fromSignIn=true&amp;trk=cold_join_sign_in")
        driver.implicitly_wait(5)
        #login
        login_bt = driver.find_element_by_xpath('//input[@name="session_key"]')
        login_bt.send_keys(login)
        #password
        password_bt = driver.find_element_by_xpath('//input[@name="session_password"]')
        password_bt.send_keys(password)
        send_button = driver.find_element_by_xpath('//div[@class="login__form_action_container "]/button')
        send_button.click()
        
        
    def search_skills(self, job, driver):
        #go to EMSI
        driver.get('https://match.emsiskills.com/skills/job')
        
        time.sleep(5)
        
        # locate search bar
        search_query = driver.find_element_by_xpath('//*[@id="skill-page-input"]')

        # Send the job title
        search_query.send_keys(job)
        
        time.sleep(3)
        
        #Arrow down
        search_query.send_keys(Keys.DOWN)
        
        time.sleep(3)

        # .send_keys() to simulate the return key 
        search_query.send_keys(Keys.ENTER)
        
        time.sleep(3)
        
        
        #Function to scrap the skills
        def get_skills():
            skills = []
            skill_elements = driver.find_elements_by_xpath("//div[@class='sc-cZBZkQ ljywcD']")

            for element in skill_elements:
                try:
                    skills.append(element.text)
                except Exception as e:
                    print(e)
            return skills
                  
        #List with all the skills
        skills = []
        
        have_next_page = True
        
        while have_next_page:
            time.sleep(3)
            #next page button
            next_button = driver.find_element_by_xpath("//button[@class='sc-ekulBa eDpOAz'][2]")
            
            #if its enabled
            if next_button.is_enabled():
                #get the skills and append to the list
                for j in get_skills():
                    skills.append(j)
                    
                #click on next button
                next_button.click()
            
            #if not enabled
            else:
                #get the skills and append to the list
                for j in get_skills():
                    skills.append(j)
                    
                #set button controller to false
                have_next_page = False

        return skills
    
    def get_job_titles(self, text, driver):
        time.sleep(1)
        #go to EMSI
        driver.get('https://match.emsiskills.com/skills/job')
        time.sleep(5)
        input_jobtitle = driver.find_element_by_xpath('//*[@id="skill-page-input"]')
        time.sleep(2)
        input_jobtitle.clear()
        input_jobtitle.send_keys(text)
        time.sleep(2)
        job_list = driver.find_element_by_id("downshift-0-menu").text
        job_list = job_list.split("\n")
        driver.close()
        return job_list
        

In [2]:
#instanciate Company class
emsi = EMSI_skills()

In [3]:
alfabeto = [chr(x) for x in range(ord('a'), ord('z')+1)]

In [ ]:
#With one letter
problems = []
job_titles = []
for i in tqdm(alfabeto):
    try:
        driver = emsi.open_driver(path="driver/chromedriver",headless=False,incognito=True)
        for job in emsi.get_job_titles(i, driver):
            job_titles.append(job)
    except:
        problems.append(i)

  4%|▍         | 1/26 [00:33<13:48, 33.14s/it]

In [ ]:
#With two letters
problems = []
job_titles = []
for i in tqdm(alfabeto):
    for j in alfabeto:
        try:
            driver = emsi.open_driver(path="driver/chromedriver",headless=False,incognito=True)
            print(i+j)
            job_titles.append(emsi.get_job_titles(i+j, driver))
        except:
            problems.append(i+j)

  0%|          | 0/26 [00:00<?, ?it/s]

aa


In [12]:
import pandas as pd
df = pd.DataFrame()
df["titles"] = job_titles
df.to_pickle("jobs.pkl")

NameError: name 'job_titles' is not defined

In [56]:
df = pd.read_pickle("jobs.pkl")
df

,titles
0,Academic Advisor
1,Account Administrator
2,Academic Affairs Specialist
3,Abstractor
4,Accompanist
...,...
449,Transportation Manager (Management)
450,Payroll Manager (Management)
451,Medical Center Manager
452,Laboratory Manager (Management)


In [ ]:
driver = emsi.open_driver(path="driver/chromedriver",headless=False,incognito=False)
job = []
skills_list = []

for i in tqdm(df["titles"]):
    job.append(i)
    skills_list.append(emsi.search_skills(i,driver))
    
    


  0%|          | 0/454 [00:00<?, ?it/s]